<a href="https://colab.research.google.com/github/ojuskumar/Stock-Forecaster/blob/main/STOCK_PICKER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from nsepy import get_history
import streamlit as st
import matplotlib.pyplot as plt

class Stock:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.historical_prices = self.download_historical_prices()

    def download_historical_prices(self):
        data = get_history(symbol=self.symbol, start=self.start_date, end=self.end_date, index=True)
        return data['Close']

    def n_day_ret(self, N, cur_date):
        return (self.historical_prices[cur_date] / self.historical_prices[cur_date - pd.DateOffset(days=N)] - 1) * 100

class Portfolio:
    def __init__(self, benchmark, stocks):
        self.benchmark = benchmark
        self.stocks = stocks

    def active_stock_selection_strategy(self, cur_date):
        selected_stocks = []
        for stock in self.stocks:
            ret = stock.n_day_ret(30, cur_date)
            if ret > 0:
                selected_stocks.append(stock.symbol)
        return selected_stocks

    def compare_performance_with_benchmark(self):
        benchmark_returns = self.benchmark.historical_prices.pct_change() * 100
        strategy_returns = pd.Series(index=self.benchmark.historical_prices.index)

        for cur_date in self.benchmark.historical_prices.index:
            selected_stocks = self.active_stock_selection_strategy(cur_date)
            strategy_returns[cur_date] = self.calculate_strategy_return(selected_stocks, cur_date)

        return benchmark_returns, strategy_returns

    def calculate_strategy_return(self, selected_stocks, cur_date):
        return np.mean([stock.n_day_ret(30, cur_date) for stock in self.stocks if stock.symbol in selected_stocks])

    def summarize_performance(self, benchmark_returns, strategy_returns):
        benchmark_cagr = ((1 + benchmark_returns.mean() / 100) ** 252 - 1) * 100
        strategy_cagr = ((1 + strategy_returns.mean() / 100) ** 252 - 1) * 100
        benchmark_volatility = benchmark_returns.std() * np.sqrt(252)
        strategy_volatility = strategy_returns.std() * np.sqrt(252)
        benchmark_sharpe_ratio = benchmark_cagr / benchmark_volatility
        strategy_sharpe_ratio = strategy_cagr / strategy_volatility

        return {
            'Benchmark CAGR (%)': benchmark_cagr,
            'Strategy CAGR (%)': strategy_cagr,
            'Benchmark Volatility (%)': benchmark_volatility,
            'Strategy Volatility (%)': strategy_volatility,
            'Benchmark Sharpe Ratio': benchmark_sharpe_ratio,
            'Strategy Sharpe Ratio': strategy_sharpe_ratio
        }

class PerformanceApp:
    def __init__(self, start_date, end_date, stocks, benchmark_symbol):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.stocks = stocks
        self.benchmark_symbol = benchmark_symbol

    def run(self):
        benchmark = Stock(self.benchmark_symbol, self.start_date, self.end_date)
        portfolio = Portfolio(benchmark, self.stocks)

        benchmark_returns, strategy_returns = portfolio.compare_performance_with_benchmark()
        performance_metrics = portfolio.summarize_performance(benchmark_returns, strategy_returns)

        plt.figure(figsize=(10, 6))
        plt.plot(benchmark_returns.cumsum(), label='Benchmark')
        plt.plot(strategy_returns.cumsum(), label='Strategy')
        plt.title('Cumulative Returns')
        plt.legend()
        st.pyplot()

        st.write("Selected Stocks for the Active Strategy:")
        st.write(portfolio.active_stock_selection_strategy(self.end_date))

        st.write("Performance Metrics:")
        st.write(performance_metrics)

# Example Usage
if __name__ == '__main__':
    start_date = '2023-01-01'
    end_date = '2024-01-01'
    benchmark_symbol = '^NSEI'
    stock_symbols = ['TCS', 'RELIANCE', 'INFY']

    stocks = [Stock(symbol, start_date, end_date) for symbol in stock_symbols]

    app = PerformanceApp(start_date, end_date, stocks, benchmark_symbol)
    app.run()



Exception in thread Thread-13 (get_history):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 132, in run
    self.result = self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 138, in get_history
    return get_history_quanta(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 143, in get_history_quanta
    df = url_to_df(url=url,
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 151, in url_to_df
    resp = url(**params)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 167, in __call__
    return self.session.get(url, params=kwargs, proxies = self.proxy )
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
  File "/usr/local/

AttributeError: ignored

In [ ]:
!pip install nsepy

  Preparing metadata (setup.py) ... done
  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36057 sha256=539f42be92b75977a106764c8599cc8bac506433786b6d509161f39a4e5e46e3
  Stored in directory: /root/.cache/pip/wheels/4c/87/cb/acaf83f625e5fc73e1fe6e2a8e97680c74cd72391850ef5a86
Successfully built nsepy


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from nsepy import get_history
import streamlit as st
import matplotlib.pyplot as plt

class Stock:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.historical_prices = self.download_historical_prices()

    def download_historical_prices(self):
        data = get_history(symbol=self.symbol, start=self.start_date, end=self.end_date, index=True)
        return data['Close']

    def n_day_ret(self, N, cur_date):
        return (self.historical_prices[cur_date] / self.historical_prices[cur_date - pd.DateOffset(days=N)] - 1) * 100

class Portfolio:
    def __init__(self, benchmark, stocks):
        self.benchmark = benchmark
        self.stocks = stocks

    def active_stock_selection_strategy(self, cur_date):
        selected_stocks = []
        for stock in self.stocks:
            ret = stock.n_day_ret(30, cur_date)
            if ret > 0:
                selected_stocks.append(stock.symbol)
        return selected_stocks

    def compare_performance_with_benchmark(self):
        benchmark_returns = self.benchmark.historical_prices.pct_change() * 100
        strategy_returns = pd.Series(index=self.benchmark.historical_prices.index)

        for cur_date in self.benchmark.historical_prices.index:
            selected_stocks = self.active_stock_selection_strategy(cur_date)
            strategy_returns[cur_date] = self.calculate_strategy_return(selected_stocks, cur_date)

        return benchmark_returns, strategy_returns

    def calculate_strategy_return(self, selected_stocks, cur_date):
        return np.mean([stock.n_day_ret(30, cur_date) for stock in self.stocks if stock.symbol in selected_stocks])

    def summarize_performance(self, benchmark_returns, strategy_returns):
        benchmark_cagr = ((1 + benchmark_returns.mean() / 100) ** 252 - 1) * 100
        strategy_cagr = ((1 + strategy_returns.mean() / 100) ** 252 - 1) * 100
        benchmark_volatility = benchmark_returns.std() * np.sqrt(252)
        strategy_volatility = strategy_returns.std() * np.sqrt(252)
        benchmark_sharpe_ratio = benchmark_cagr / benchmark_volatility
        strategy_sharpe_ratio = strategy_cagr / strategy_volatility

        return {
            'Benchmark CAGR (%)': benchmark_cagr,
            'Strategy CAGR (%)': strategy_cagr,
            'Benchmark Volatility (%)': benchmark_volatility,
            'Strategy Volatility (%)': strategy_volatility,
            'Benchmark Sharpe Ratio': benchmark_sharpe_ratio,
            'Strategy Sharpe Ratio': strategy_sharpe_ratio
        }

class PerformanceApp:
    def __init__(self, start_date, end_date, stocks, benchmark_symbol):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.stocks = stocks
        self.benchmark_symbol = benchmark_symbol

    def run(self):
        benchmark = Stock(self.benchmark_symbol, self.start_date, self.end_date)
        portfolio = Portfolio(benchmark, self.stocks)

        benchmark_returns, strategy_returns = portfolio.compare_performance_with_benchmark()
        performance_metrics = portfolio.summarize_performance(benchmark_returns, strategy_returns)

        plt.figure(figsize=(10, 6))
        plt.plot(benchmark_returns.cumsum(), label='Benchmark')
        plt.plot(strategy_returns.cumsum(), label='Strategy')
        plt.title('Cumulative Returns')
        plt.legend()
        st.pyplot(plt.gcf())  # Change this line

        st.write("Selected Stocks for the Active Strategy:")
        st.write(portfolio.active_stock_selection_strategy(self.end_date))

        st.write("Performance Metrics:")
        st.write(performance_metrics)

# Example Usage
if __name__ == '__main__':
    start_date = '2023-01-01'
    end_date = '2024-01-01'
    benchmark_symbol = '^NSEI'
    stock_symbols = ['TCS', 'RELIANCE', 'INFY']

    stocks = [Stock(symbol, start_date, end_date) for symbol in stock_symbols]

    app = PerformanceApp(start_date, end_date, stocks, benchmark_symbol)
    app.run()


Exception in thread Thread-17 (get_history):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 132, in run
    self.result = self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 138, in get_history
    return get_history_quanta(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 143, in get_history_quanta
    df = url_to_df(url=url,
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 151, in url_to_df
    resp = url(**params)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 167, in __call__
    return self.session.get(url, params=kwargs, proxies = self.proxy )
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
  File "/usr/local/

AttributeError: ignored

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from nsepy import get_history
import matplotlib.pyplot as plt

class Stock:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.historical_prices = self.download_historical_prices()

    def download_historical_prices(self):
        data = get_history(symbol=self.symbol, start=self.start_date, end=self.end_date, index=True)
        return data['Close']

    def n_day_ret(self, N, cur_date):
        return (self.historical_prices[cur_date] / self.historical_prices[cur_date - pd.DateOffset(days=N)] - 1) * 100

class Portfolio:
    def __init__(self, benchmark, stocks):
        self.benchmark = benchmark
        self.stocks = stocks

    def active_stock_selection_strategy(self, cur_date):
        selected_stocks = []
        for stock in self.stocks:
            ret = stock.n_day_ret(30, cur_date)
            if ret > 0:
                selected_stocks.append(stock.symbol)
        return selected_stocks

    def compare_performance_with_benchmark(self):
        benchmark_returns = self.benchmark.historical_prices.pct_change() * 100
        strategy_returns = pd.Series(index=self.benchmark.historical_prices.index)

        for cur_date in self.benchmark.historical_prices.index:
            selected_stocks = self.active_stock_selection_strategy(cur_date)
            strategy_returns[cur_date] = self.calculate_strategy_return(selected_stocks, cur_date)

        return benchmark_returns, strategy_returns

    def calculate_strategy_return(self, selected_stocks, cur_date):
        return np.mean([stock.n_day_ret(30, cur_date) for stock in self.stocks if stock.symbol in selected_stocks])

    def summarize_performance(self, benchmark_returns, strategy_returns):
        benchmark_cagr = ((1 + benchmark_returns.mean() / 100) ** 252 - 1) * 100
        strategy_cagr = ((1 + strategy_returns.mean() / 100) ** 252 - 1) * 100
        benchmark_volatility = benchmark_returns.std() * np.sqrt(252)
        strategy_volatility = strategy_returns.std() * np.sqrt(252)
        benchmark_sharpe_ratio = benchmark_cagr / benchmark_volatility
        strategy_sharpe_ratio = strategy_cagr / strategy_volatility

        return {
            'Benchmark CAGR (%)': benchmark_cagr,
            'Strategy CAGR (%)': strategy_cagr,
            'Benchmark Volatility (%)': benchmark_volatility,
            'Strategy Volatility (%)': strategy_volatility,
            'Benchmark Sharpe Ratio': benchmark_sharpe_ratio,
            'Strategy Sharpe Ratio': strategy_sharpe_ratio
        }

# Example Usage
start_date = '2023-01-01'
end_date = '2024-01-01'
benchmark_symbol = '^NSEI'
stock_symbols = ['TCS', 'RELIANCE', 'INFY']

stocks = [Stock(symbol, start_date, end_date) for symbol in stock_symbols]

portfolio = Portfolio(Stock(benchmark_symbol, start_date, end_date), stocks)

benchmark_returns, strategy_returns = portfolio.compare_performance_with_benchmark()

# Plotting Cumulative Returns
plt.figure(figsize=(10, 6))
plt.plot(benchmark_returns.cumsum(), label='Benchmark')
plt.plot(strategy_returns.cumsum(), label='Strategy')
plt.title('Cumulative Returns')
plt.legend()
plt.show()


Exception in thread Thread-18 (get_history):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 132, in run
    self.result = self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 138, in get_history
    return get_history_quanta(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 143, in get_history_quanta
    df = url_to_df(url=url,
  File "/usr/local/lib/python3.10/dist-packages/nsepy/history.py", line 151, in url_to_df
    resp = url(**params)
  File "/usr/local/lib/python3.10/dist-packages/nsepy/commons.py", line 167, in __call__
    return self.session.get(url, params=kwargs, proxies = self.proxy )
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
  File "/usr/local/

AttributeError: ignored

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

class Stock:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.historical_prices = self.download_historical_prices()

    def download_historical_prices(self):
        data = yf.download(self.symbol, start=self.start_date, end=self.end_date)
        return data['Close']

    def n_day_ret(self, N, cur_date):
        return (self.historical_prices.loc[cur_date] / self.historical_prices.loc[cur_date - pd.DateOffset(days=N)] - 1) * 100

class Portfolio:
    def __init__(self, benchmark, stocks):
        self.benchmark = benchmark
        self.stocks = stocks

    def active_stock_selection_strategy(self, cur_date):
        selected_stocks = []
        for stock in self.stocks:
            ret = stock.n_day_ret(30, cur_date)
            if ret > 0:
                selected_stocks.append(stock.symbol)
        return selected_stocks

    def compare_performance_with_benchmark(self):
        benchmark_returns = self.benchmark.historical_prices.pct_change() * 100
        strategy_returns = pd.Series(index=self.benchmark.historical_prices.index)

        for cur_date in self.benchmark.historical_prices.index:
            selected_stocks = self.active_stock_selection_strategy(cur_date)
            strategy_returns.loc[cur_date] = self.calculate_strategy_return(selected_stocks, cur_date)

        return benchmark_returns, strategy_returns

    def calculate_strategy_return(self, selected_stocks, cur_date):
        return np.mean([stock.n_day_ret(30, cur_date) for stock in self.stocks if stock.symbol in selected_stocks])

    def summarize_performance(self, benchmark_returns, strategy_returns):
        benchmark_cagr = ((1 + benchmark_returns.mean() / 100) ** 252 - 1) * 100
        strategy_cagr = ((1 + strategy_returns.mean() / 100) ** 252 - 1) * 100
        benchmark_volatility = benchmark_returns.std() * np.sqrt(252)
        strategy_volatility = strategy_returns.std() * np.sqrt(252)
        benchmark_sharpe_ratio = benchmark_cagr / benchmark_volatility
        strategy_sharpe_ratio = strategy_cagr / strategy_volatility

        return {
            'Benchmark CAGR (%)': benchmark_cagr,
            'Strategy CAGR (%)': strategy_cagr,
            'Benchmark Volatility (%)': benchmark_volatility,
            'Strategy Volatility (%)': strategy_volatility,
            'Benchmark Sharpe Ratio': benchmark_sharpe_ratio,
            'Strategy Sharpe Ratio': strategy_sharpe_ratio
        }

# Example Usage
start_date = '2023-01-01'
end_date = '2024-01-01'
benchmark_symbol = '^NSEI'
stock_symbols = ['TCS.BO', 'RELIANCE.BO', 'INFY.BO']  # Yahoo Finance stock symbols

stocks = [Stock(symbol, start_date, end_date) for symbol in stock_symbols]

portfolio = Portfolio(Stock(benchmark_symbol, start_date, end_date), stocks)

benchmark_returns, strategy_returns = portfolio.compare_performance_with_benchmark()

# Plotting Cumulative Returns
plt.figure(figsize=(10, 6))
plt.plot(benchmark_returns.cumsum(), label='Benchmark')
plt.plot(strategy_returns.cumsum(), label='Strategy')
plt.title('Cumulative Returns')
plt.legend()
plt.show()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


<ipython-input-11-393128a3109e>:35: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  strategy_returns = pd.Series(index=self.benchmark.historical_prices.index)


KeyError: ignored